In [3]:
import os
import glob
import rasterio
from rasterio.merge import merge
import numpy as np

def mean_merger(arrays, *args, **kwargs):
    """
    Custom callable for rasterio.merge.merge's 'method' parameter.
    Calculates the mean of overlapping pixels from a masked array and
    ignores any extra positional or keyword arguments passed by the merge function.
    
    Args:
        arrays (numpy.ma.MaskedArray): A masked array where valid data pixels
                                       are unmasked. This is the first positional argument.
        *args: Catches extra positional arguments (like out_shape, out_transform) and ignores them.
        **kwargs: Catches any other keyword arguments like 'index' and ignores them.

    Returns:
        numpy.ndarray: An array with the mean values, with masked values handled.
    """
    # The `mean` method of a NumPy masked array correctly ignores the masked (NoData) values.
    # We calculate the mean along the first axis (the axis that stacks the different rasters).
    # The result is a simple ndarray, with the mask applied during calculation.
    return np.ma.mean(arrays, axis=0)


def merge_raster_files(input_dir, output_file, method='first', nodata_val=-9999.0, output_dtype='float32'):
    """
    Merges all GeoTIFF raster files in a directory into a single raster.

    Args:
        input_dir (str): The directory containing the raster files to merge.
        output_file (str): The full path for the output merged raster file.
        method (str or callable): The method to use for overlapping areas. Options include
                                  'first', 'last', 'min', 'max', 'sum', 'count', or
                                  a callable function like our mean_merger.
        nodata_val (float/int): The NoData value to use for the output raster.
        output_dtype (str): The desired data type for the output raster (e.g., 'float32', 'int16').
    """
    # Use the function's name for printing if a callable is provided
    method_name = method if isinstance(method, str) else method.__name__
    
    print(f"\n--- Starting Merge Process ---")
    print(f"Input directory: {input_dir}")
    print(f"Output file: {output_file}")
    print(f"Overlap method: {method_name}")

    try:
        # Find all GeoTIFF files in the input directory (case-insensitive search for .tif/.tiff)
        search_criteria_lower = os.path.join(input_dir, '*.tif')
        search_criteria_upper = os.path.join(input_dir, '*.TIF')
        search_criteria_tiff_lower = os.path.join(input_dir, '*.tiff')
        search_criteria_tiff_upper = os.path.join(input_dir, '*.TIFF')
        
        raster_files_to_merge = glob.glob(search_criteria_lower) + \
                                glob.glob(search_criteria_upper) + \
                                glob.glob(search_criteria_tiff_lower) + \
                                glob.glob(search_criteria_tiff_upper)

        if not raster_files_to_merge:
            print(f"!!! No .tif or .tiff files found in '{input_dir}'. Please check the path.")
            return

        print(f"Found {len(raster_files_to_merge)} raster files to merge.")

        # Open all raster files
        src_files_to_mosaic = [rasterio.open(fp) for fp in raster_files_to_merge]

        # Merge the rasters
        # The 'method' argument can be a string or a callable function
        mosaic, out_trans = merge(src_files_to_mosaic, method=method, nodata=nodata_val)

        # Get the metadata from the first raster and update it for the output
        out_meta = src_files_to_mosaic[0].meta.copy()
        
        # Update metadata properties for the output raster
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans,
            "crs": src_files_to_mosaic[0].crs, # Assumes all inputs have the same CRS
            "dtype": output_dtype,
            "nodata": nodata_val,
            "compress": "lzw" # Add LZW compression for smaller file size
        })

        # Write the mosaicked raster to the output file
        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(mosaic.astype(output_dtype))

        # Close all the source files
        for src in src_files_to_mosaic:
            src.close()
            
        print(f"Successfully merged files and saved to: {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# --- MAIN SCRIPT EXECUTION ---
if __name__ == "__main__":
    # --- USER: SET YOUR PATHS AND SETTINGS HERE ---

    # 1. Settings for Merging DEM files
    dem_input_directory = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/DEM"
    dem_output_file = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final.tif"
    
    # 2. Settings for Merging NDVI files
    ndvi_input_directory = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/LANDSAT/NDVI/NDVI_OUTPUT"
    ndvi_output_file = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/NDVI_MERGED.tif"
    
    # --- END OF USER SETTINGS ---

    # --- Run the Merge for DEM Files ---
    # For DEM tiles, 'first' is usually sufficient as they are designed to be adjacent.
    print("--- Processing Digital Elevation Model (DEM) ---")
    merge_raster_files(
        input_dir=dem_input_directory,
        output_file=dem_output_file,
        method='first', 
        nodata_val=-32768, # A common NoData value for SRTM DEM
        output_dtype='int16' # SRTM elevation is integer data
    )

    # --- Run the Merge for NDVI Files ---
    # We now pass our custom 'mean_merger' function as the method. This function
    # will correctly calculate the mean of overlapping pixels while ignoring extra
    # arguments like 'index' and other positional arguments that caused the previous errors.
    print("\n\n--- Processing Normalized Difference Vegetation Index (NDVI) ---")
    merge_raster_files(
        input_dir=ndvi_input_directory,
        output_file=ndvi_output_file,
        method=mean_merger, # Pass our custom function
        nodata_val=-9999.0,
        output_dtype='float32' # NDVI requires floating point precision
    )

    print("\n\nAll merge processes are complete.")


--- Processing Digital Elevation Model (DEM) ---

--- Starting Merge Process ---
Input directory: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/DEM
Output file: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final.tif
Overlap method: first
Found 2 raster files to merge.
Successfully merged files and saved to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final.tif


--- Processing Normalized Difference Vegetation Index (NDVI) ---

--- Starting Merge Process ---
Input directory: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/LANDSAT/NDVI/NDVI_OUTPUT
Output file: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/NDVI_MERGED.tif
Overlap method: mean_merger
Found 2 raster files to merge.
Successfully merged files and saved to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/NDVI_MERGED.tif


All merge processes are complete.


In [4]:
import os
import glob
import rasterio
from rasterio.merge import merge
import numpy as np

def mean_merger(arrays, *args, **kwargs):
    """
    Custom callable for rasterio.merge.merge's 'method' parameter.
    Calculates the mean of overlapping pixels from a masked array and
    ignores any extra positional or keyword arguments passed by the merge function.
    """
    return np.ma.mean(arrays, axis=0)


def merge_raster_files(input_dir, output_file, method='first', nodata_val=-9999.0, output_dtype='float32'):
    """
    Merges all GeoTIFF raster files in a directory into a single raster.
    Includes print statements to show which files are being merged.
    """
    method_name = method if isinstance(method, str) else method.__name__
    
    print(f"\n--- Starting Merge Process ---")
    print(f"Input directory: {input_dir}")
    print(f"Output file: {output_file}")
    print(f"Overlap method: {method_name}")

    try:
        search_criteria_lower = os.path.join(input_dir, '*.tif')
        search_criteria_upper = os.path.join(input_dir, '*.TIF')
        search_criteria_tiff_lower = os.path.join(input_dir, '*.tiff')
        search_criteria_tiff_upper = os.path.join(input_dir, '*.TIFF')
        
        raster_files_to_merge = glob.glob(search_criteria_lower) + \
                                glob.glob(search_criteria_upper) + \
                                glob.glob(search_criteria_tiff_lower) + \
                                glob.glob(search_criteria_tiff_upper)

        if not raster_files_to_merge:
            print(f"!!! No .tif or .tiff files found in '{input_dir}'. Please check the path.")
            return

        print(f"Found {len(raster_files_to_merge)} raster files to merge.")
        
        # --- ADDED DEBUG PRINT: List the files being merged ---
        print("Files being merged:")
        for f_path in raster_files_to_merge:
            print(f"  - {os.path.basename(f_path)}")
        # --- END ADDED DEBUG PRINT ---

        src_files_to_mosaic = [rasterio.open(fp) for fp in raster_files_to_merge]
        mosaic, out_trans = merge(src_files_to_mosaic, method=method, nodata=nodata_val)
        out_meta = src_files_to_mosaic[0].meta.copy()
        
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans,
            "crs": src_files_to_mosaic[0].crs, 
            "dtype": output_dtype,
            "nodata": nodata_val,
            "compress": "lzw" 
        })

        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(mosaic.astype(output_dtype))

        for src in src_files_to_mosaic:
            src.close()
            
        print(f"Successfully merged files and saved to: {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# --- MAIN SCRIPT EXECUTION ---
if __name__ == "__main__":
    # --- USER: SET YOUR PATHS AND SETTINGS HERE ---

    # 1. Settings for Merging DEM files
    dem_input_directory = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/DEM"
    dem_output_file = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final.tif"
    
    # 2. Settings for Merging NDVI files
    ndvi_input_directory = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/LANDSAT/NDVI/NDVI_OUTPUT"
    ndvi_output_file = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/NDVI_MERGED.tif"
    
    # --- END OF USER SETTINGS ---

    # --- Run the Merge for DEM Files ---
    print("--- Processing Digital Elevation Model (DEM) ---")
    merge_raster_files(
        input_dir=dem_input_directory,
        output_file=dem_output_file,
        method='first', 
        nodata_val=-32768, 
        output_dtype='int16' 
    )

    # --- Run the Merge for NDVI Files ---
    print("\n\n--- Processing Normalized Difference Vegetation Index (NDVI) ---")
    merge_raster_files(
        input_dir=ndvi_input_directory,
        output_file=ndvi_output_file,
        method=mean_merger, 
        nodata_val=-9999.0,
        output_dtype='float32' 
    )

    print("\n\nAll merge processes are complete.")

--- Processing Digital Elevation Model (DEM) ---

--- Starting Merge Process ---
Input directory: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/DEM
Output file: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final.tif
Overlap method: first
Found 2 raster files to merge.
Files being merged:
  - n17_e073_1arc_v3.tif
  - n17_e073_1arc_v3.tif
Successfully merged files and saved to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final.tif


--- Processing Normalized Difference Vegetation Index (NDVI) ---

--- Starting Merge Process ---
Input directory: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/LANDSAT/NDVI/NDVI_OUTPUT
Output file: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/NDVI_MERGED.tif
Overlap method: mean_merger
Found 2 raster files to merge.
Files being merged:
  - NDVI_(LC08_L1TP_147048_20241228_20250104_02_T1).TIF
  - NDVI_(LC08_L1TP_147048_20241228_20250

In [6]:
import os
import glob
import rasterio
from rasterio.merge import merge
import numpy as np

def mean_merger(arrays, *args, **kwargs):
    """
    Custom callable for rasterio.merge.merge's 'method' parameter.
    Calculates the mean of overlapping pixels from a masked array and
    ignores any extra positional or keyword arguments passed by the merge function.
    """
    return np.ma.mean(arrays, axis=0)


def merge_raster_files(input_dir, output_file, method='first', nodata_val=-9999.0, output_dtype='float32', specific_filenames=None):
    """
    Merges specified GeoTIFF raster files in a directory into a single raster.
    If specific_filenames is provided, it will only merge those files.
    """
    method_name = method if isinstance(method, str) else method.__name__
    
    print(f"\n--- Starting Merge Process ---")
    print(f"Input directory: {input_dir}")
    print(f"Output file: {output_file}")
    print(f"Overlap method: {method_name}")

    try:
        raster_files_to_merge = []
        if specific_filenames:
            # If specific filenames are provided, only look for those
            for fname in specific_filenames:
                fpath = os.path.join(input_dir, fname)
                if os.path.exists(fpath):
                    raster_files_to_merge.append(fpath)
                else:
                    print(f"!!! Warning: Specified file not found: {fpath}. Skipping.")
        else:
            # Otherwise, use the generic glob search (original behavior)
            search_criteria_lower = os.path.join(input_dir, '*.tif')
            search_criteria_upper = os.path.join(input_dir, '*.TIF')
            search_criteria_tiff_lower = os.path.join(input_dir, '*.tiff')
            search_criteria_tiff_upper = os.path.join(input_dir, '*.TIFF')
            
            raster_files_to_merge = glob.glob(search_criteria_lower) + \
                                    glob.glob(search_criteria_upper) + \
                                    glob.glob(search_criteria_tiff_lower) + \
                                    glob.glob(search_criteria_tiff_upper)
            raster_files_to_merge = list(set(raster_files_to_merge)) # Remove duplicates from generic search


        if not raster_files_to_merge:
            print(f"!!! No raster files found in '{input_dir}' for merging. Please check the path or specified filenames.")
            return

        print(f"Found {len(raster_files_to_merge)} raster files to merge.")
        
        print("Files being merged:")
        for f_path in raster_files_to_merge:
            print(f"  - {os.path.basename(f_path)}")

        src_files_to_mosaic = [rasterio.open(fp) for fp in raster_files_to_merge]
        mosaic, out_trans = merge(src_files_to_mosaic, method=method, nodata=nodata_val)
        out_meta = src_files_to_mosaic[0].meta.copy()
        
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans,
            "crs": src_files_to_mosaic[0].crs, 
            "dtype": output_dtype,
            "nodata": nodata_val,
            "compress": "lzw" 
        })

        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(mosaic.astype(output_dtype))

        for src in src_files_to_mosaic:
            src.close()
            
        print(f"Successfully merged files and saved to: {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# --- MAIN SCRIPT EXECUTION ---
if __name__ == "__main__":
    # --- USER: SET YOUR PATHS AND SETTINGS HERE ---

    # 1. Settings for Merging DEM files
    dem_input_directory = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/DEM"
    dem_output_file = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final.tif"
    # Specify the exact filename of your DEM if you only have one and want to avoid glob issues
    # REPLACE 'your_dem_filename.tif' with the actual name, e.g., 'n17_e073_1arc_v3.tif'
    specific_dem_filename = 'n17_e073_1arc_v3.tif' 
    
    # 2. Settings for Merging NDVI files
    ndvi_input_directory = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/LANDSAT/NDVI/NDVI_OUTPUT"
    ndvi_output_file = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/NDVI_MERGED.tif"
    # Specify the exact filename of your NDVI output file
    # REPLACE 'your_ndvi_filename.TIF' with the actual name, e.g., 'NDVI_(LC08_L1TP_147048_20241228_20250104_02_T1).TIF'
    specific_ndvi_filename = 'NDVI_(LC08_L1TP_147048_20241228_20250104_02_T1).TIF'
    
    # --- END OF USER SETTINGS ---

    # --- Run the Merge for DEM Files ---
    print("--- Processing Digital Elevation Model (DEM) ---")
    merge_raster_files(
        input_dir=dem_input_directory,
        output_file=dem_output_file,
        method='first', 
        nodata_val=-32768, 
        output_dtype='int16',
        specific_filenames=[specific_dem_filename] # Pass the specific filename here
    )

    # --- Run the Merge for NDVI Files ---
    print("\n\n--- Processing Normalized Difference Vegetation Index (NDVI) ---")
    merge_raster_files(
        input_dir=ndvi_input_directory,
        output_file=ndvi_output_file,
        method=mean_merger, 
        nodata_val=-9999.0,
        output_dtype='float32',
        specific_filenames=[specific_ndvi_filename] # Pass the specific filename here
    )

    print("\n\nAll merge processes are complete.")

--- Processing Digital Elevation Model (DEM) ---

--- Starting Merge Process ---
Input directory: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/DEM
Output file: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final.tif
Overlap method: first
Found 1 raster files to merge.
Files being merged:
  - n17_e073_1arc_v3.tif
Successfully merged files and saved to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/DEM_Merged_Final.tif


--- Processing Normalized Difference Vegetation Index (NDVI) ---

--- Starting Merge Process ---
Input directory: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/LANDSAT/NDVI/NDVI_OUTPUT
Output file: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/NDVI_MERGED.tif
Overlap method: mean_merger
Found 1 raster files to merge.
Files being merged:
  - NDVI_(LC08_L1TP_147048_20241228_20250104_02_T1).TIF
Successfully merged files and saved to: C:/Users/Lenovo/Documents/